In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import b3d
import sys
sys.path.append(b3d.get_root_path())

In [3]:
from tests.patch_tracking.task import PatchTrackingTask
import rerun as rr

In [16]:
spec = PatchTrackingTask.get_known_unity_scene_specs()[0]
task = PatchTrackingTask.task_from_known_unity_scene_spec(spec, n_frames=5)


In [19]:
rr.init(f'task--{spec["scene_name"]}-2')
rr.connect("127.0.0.1:8812")
task.visualize_task()

In [21]:
task_spec = task.get_task_specification()

In [22]:
from tests.patch_tracking.solver import AdamPatchTracker
solver = AdamPatchTracker()
solution = solver.solve(task_spec)

In [23]:
solver.visualize_solver_state(task_spec)

/home/georgematheos/b3d/b3d/chisight/dense/model.py:124: RerunWarning: Pinhole: ValueError(Expected focal_length to be one or two floats)
  rr.Pinhole(


In [53]:
video = task_spec["video"]
assert video.shape[-1] == 4, "Expected video to be RGBD"
X_WC = task_spec["camera_pose"]
initial_patch_centers_2D = task_spec["initial_patch_positions_2D"]
r = task_spec["renderer"]
fx, fy, cx, cy = r.fx, r.fy, r.cx, r.cy

# (patch_vertices_P, patch_faces, patch_vertex_colors, Xs_WP, _) = b3d.chisight.dense.patch_tracking.get_patches(
#     initial_patch_centers_2D, video, X_WC, fx, fy, cx, cy
# )
rgbds = video
depths = rgbds[..., 3]
xyzs_C = b3d.utils.xyz_from_depth_vectorized(depths, fx, fy, cx, cy)
xyzs_W = X_WC.apply(xyzs_C)

In [54]:
import jax.numpy as jnp
xyzs_C = X_WC.inv().apply(xyzs_W)

centers = initial_patch_centers_2D


In [55]:
pairwise_euclidean_dists = jnp.linalg.norm(centers[:, None] - centers[None], axis=-1)
min_nonzero_dist = jnp.min(jnp.where(pairwise_euclidean_dists != 0, pairwise_euclidean_dists, jnp.inf))
del_pix = jnp.astype(jnp.ceil(min_nonzero_dist / (2 * jnp.sqrt(2))), int)
assert del_pix > 1, "Patch size is too small"
print(f"del_pix = {del_pix}")


del_pix = 4


In [57]:
center = centers[0]
center

Array([460.734 , 603.7762], dtype=float32)

In [60]:
jnp.round(center)

Array([461., 604.], dtype=float32)

In [59]:
import jax
center_x, center_y = center[0], center[1]
patch_points_C = jax.lax.dynamic_slice(xyzs_C[0], (center_x-del_pix,center_y-del_pix,0), (2*del_pix-1,2*del_pix-1,3)).reshape(-1,3)


TypeError: index arguments to dynamic_slice must be integers of the same type, got: float32, float32, int32

In [19]:
pairwise_x_distances = jnp.abs(centers[:, 0] - centers[:, 0, None])
pairwise_y_distances = jnp.abs(centers[:, 1] - centers[:, 1, None])
pairwise_xymin_distances = jnp.min(jnp.array([pairwise_x_distances, pairwise_y_distances]), axis=0)

min_nonzero_x_distance = jnp.min(jnp.where(pairwise_x_distances != 0, pairwise_x_distances, jnp.inf))
min_nonzero_y_distance = jnp.min(jnp.where(pairwise_y_distances != 0, pairwise_y_distances, jnp.inf))
del_pix = jnp.astype(jnp.min(jnp.array([min_nonzero_x_distance, min_nonzero_y_distance])) / 2 - 1, int)

In [26]:
pairwise_euclidean_dists = jnp.linalg.norm(centers[:, None] - centers[None], axis=-1)
min_nonzero_dist = jnp.min(jnp.where(pairwise_euclidean_dists != 0, pairwise_euclidean_dists, jnp.inf))
del_pix = min_nonzero_dist / (2 * jnp.sqrt(2))
del_pix

Array(3.538479, dtype=float32)

In [28]:
pairwise_taxicab_dists = jnp.sum(jnp.abs(centers[:, None] - centers[None]), axis=-1)
min_nonzero_dist = jnp.min(jnp.where(pairwise_taxicab_dists != 0, pairwise_taxicab_dists, jnp.inf))
min_nonzero_dist

Array(10.032928, dtype=float32)

In [50]:
jnp.ceil(2.0, dtype=int)

TypeError: Cannot interpret value of type <class 'type'> as an abstract array; it does not have a dtype attribute

In [43]:
# Compute, for each pair of centers `i, j`, the min of the x dist from i to j and the y dist from i to j
pairwise_x_distances = jnp.abs(centers[:, 0] - centers[:, 0, None])
pairwise_y_distances = jnp.abs(centers[:, 1] - centers[:, 1, None])
pairwise_xymin_distances = jnp.minimum(pairwise_x_distances, pairwise_y_distances)
min_nonzero_dist = jnp.min(jnp.where(pairwise_xymin_distances != 0, pairwise_xymin_distances, jnp.inf))
min_nonzero_dist

Array(0.00691223, dtype=float32)

In [45]:
pairwise_x_distances[:4, :4]

Array([[0.        , 8.802307  , 8.553833  , 9.190826  ],
       [8.802307  , 0.        , 0.24847412, 0.3885193 ],
       [8.553833  , 0.24847412, 0.        , 0.6369934 ],
       [9.190826  , 0.3885193 , 0.6369934 , 0.        ]], dtype=float32)

In [46]:
pairwise_y_distances[:4, :4]

Array([[ 0.       , 21.383606 , 45.431458 ,  7.2611694],
       [21.383606 ,  0.       , 24.047852 , 28.644775 ],
       [45.431458 , 24.047852 ,  0.       , 52.692627 ],
       [ 7.2611694, 28.644775 , 52.692627 ,  0.       ]], dtype=float32)

In [47]:
pairwise_xymin_distances[:4, :4]

Array([[0.        , 8.802307  , 8.553833  , 7.2611694 ],
       [8.802307  , 0.        , 0.24847412, 0.3885193 ],
       [8.553833  , 0.24847412, 0.        , 0.6369934 ],
       [7.2611694 , 0.3885193 , 0.6369934 , 0.        ]], dtype=float32)

In [39]:
pairwise_x_distances.shape

(307, 307)

In [27]:
6 * jnp.sqrt(2)

Array(8.485281, dtype=float32, weak_type=True)

In [22]:
min_nonzero_y_distance

Array(0.00691223, dtype=float32)

In [16]:
del_pix

Array(0, dtype=int32)

In [18]:
second_min_x

Array(13.528473, dtype=float32)

In [62]:
task_spec["renderer"].width

Array(800, dtype=int32)